# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [1]:
import findspark
findspark.init()
import pyspark
from pyspark import sql
from pyspark.sql import SparkSession, Row
from pyspark.sql import functions as f
# from pyspark.sql import SparkSession
from pyspark.sql.window import Window

In [2]:
sc = pyspark.SparkContext(appName='jupyter')
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-01-21 18:38:13,802 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [97]:
data = se.read.csv("hdfs:/data/clickstream.csv", header=True, sep = '\t')
data.registerTempTable("data")
df = data

In [84]:
df.show(5)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
+-------+----------+------------+----------+----------+
only showing top 5 rows



Spark DF solution

In [98]:
session = (Window
           .partitionBy("user_id", "session_id")
           .orderBy("timestamp")
          )

In [99]:
# Working variant
routes = (df
          #Adding error flog to events with errors
          .withColumn('isError', f.when(f.col('event_type').contains('error'),True))
          #Marking session with error and all the following lines after the error or at the same time with it with flog in order to remove them
          .withColumn('to_drop', f.last('isError', ignorenulls = True).over(session))
          #Removing all lines from error sessions
          .filter(f.isnull("to_drop"))
          #Marking all consecutive duplicate lines in session 
          .withColumn("duplicate", f.when(f.lag("event_page").over(session) == f.col("event_page"), True))
          #Removing all consecutive duplicate lines in session
          .filter(f.isnull("duplicate"))
          #Collect route in one column from several rows
          .withColumn('route',(f.collect_list('event_page')).over(session))
          #Joining collecting route to one string with delimiter "-"
          .withColumn("route", f.array_join(f.col("route"), "-"))
          .groupBy("user_id", "session_id")
          .agg(f.max('route').alias('route'))
          .select("route")
         )

In [116]:
result_df = (routes
          .select("route")
          .groupBy("route")
          .agg(f.count(f.col("route")).alias("count"))
          .orderBy("count",  ascending = False)
          .limit(30)
          )

In [120]:
result_df.show(35)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [162]:
result_df.rdd.map(lambda x: f"{x[0]}\t{x[1]}").coalesce(1).saveAsTextFile("/data/solution_df")

SQL solution

In [165]:
df.createOrReplaceTempView("view")

In [166]:
sql_result = se.sql(
                """
                SELECT ARRAY_JOIN(route, '-') as route, count(route) as count
                FROM
                    (SELECT user_id, session_id, MAX(route) AS route
                    FROM
                        (SELECT *,
                        --Collect route in one column from several rows
                        COLLECT_LIST(event_page) OVER(PARTITION BY user_id, session_id ORDER BY timestamp) AS route
                        FROM
                           (SELECT *
                            FROM
                                (SELECT *,
                                --Removing all consecutive duplicate lines in session
                                CASE WHEN LAG(event_page) OVER(PARTITION BY user_id, session_id ORDER BY timestamp) = event_page THEN TRUE
                                ELSE NULL
                                END AS is_duplicate
                                FROM
                                    (SELECT *
                                    FROM
                                        (SELECT *,
                                        --Marking session with error and all the following lines after the error or at the same time with it with flog in order to remove them
                                        LAST(is_error, TRUE) OVER(PARTITION BY user_id, session_id ORDER BY timestamp) AS to_drop
                                        FROM
                                            (SELECT *,
                                            --Adding error flog to events with errors
                                            CASE WHEN event_type LIKE '%error%' THEN TRUE ELSE NULL END AS is_error 
                                            FROM view
                                            ) AS vw_with_error
                                        ) AS vw_drop_marked
                                    WHERE to_drop IS NULL
                                    ) AS vw_dup_marked                    
                                 )
                                 WHERE is_duplicate IS NULL
                                 ) AS ROUTES
                            )
                            GROUP BY user_id, session_id
                         )
                    GROUP BY route
                    ORDER BY count DESC, route ASC
                    LIMIT 30
                """
                )
sql_result.show(30)

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1113|
|         main-rabota| 1047|
|       main-internet|  897|
|          main-bonus|  870|
|           main-news|  769|
|        main-tariffs|  677|
|         main-online|  587|
|          main-vklad|  518|
| main-rabota-archive|  170|
| main-archive-rabota|  167|
|  main-bonus-archive|  143|
|   main-rabota-bonus|  139|
|   main-bonus-rabota|  135|
|    main-news-rabota|  135|
|main-archive-inte...|  132|
|    main-rabota-news|  130|
|main-internet-rabota|  129|
|   main-archive-news|  126|
|main-rabota-internet|  124|
|main-internet-arc...|  123|
|  main-archive-bonus|  117|
| main-internet-bonus|  115|
|main-tariffs-inte...|  114|
|   main-news-archive|  113|
|  main-news-internet|  109|
|main-archive-tariffs|  104|
|  main-internet-news|  103|
|main-tariffs-archive|  103|
|    main-rabota-main|   94|
+--------------------+-----+



In [167]:
sql_result.rdd.map(lambda x: f"{x[0]}\t{x[1]}").coalesce(1).saveAsTextFile("/data/solution_sql")

In [170]:
from itertools import groupby 
#Function to remove duplicates and combine route to one string
def rem_dupl(lst):   
    result = '-'.join([i[0] for i in groupby(lst)])
    return result


In [171]:
rdd_data = df.rdd
#Celecting string with error into separate RDD
is_error = (rdd_data
            .filter(lambda row: 'error' in row.event_type)
            .map(lambda row: ((row.user_id, row.session_id), row.timestamp))
            .reduceByKey(lambda x, y: min(x, y))
           )

In [180]:
rdd_result =    sc.parallelize(
                rdd_data
                 #Deleting all data from sessions with errors
                .map(lambda row: ((row.user_id, row.session_id), row))
                .leftOuterJoin(is_error)
                .filter(lambda row: row[1][1] is None or row[1][0].timestamp < row[1][1])
                 #Rearranging data and combining route into one string, removing duplicates by using user function rem_dupl
                .map(lambda row: row[1][0])
                .map(lambda row: ((row.user_id, row.session_id), row.event_page))
                .map(lambda row: (row[0], [row[1]]))
                .reduceByKey(lambda x, y: x + y)
                .map(lambda row: (row[0], rem_dupl(row[1])))
                #Grouping by route and count them
                .map(lambda row: (row[1], 1))
                .reduceByKey(lambda x, y: x + y)
                .sortBy(lambda row: row[1], ascending=False)
                .take(30)
                ).cache()

In [182]:
rdd_result.map(lambda x: f"{x[0]}\t{x[1]}").coalesce(1).saveAsTextFile("/data/solution_rdd")

In [191]:
sc.stop()